In [ ]:
!pip install git+https://github.com/microsoft/recommenders.git

  Cloning https://github.com/microsoft/recommenders.git to /tmp/pip-req-build-caj8kkhm
  Running command git clone --filter=blob:none --quiet https://github.com/microsoft/recommenders.git /tmp/pip-req-build-caj8kkhm
  Resolved https://github.com/microsoft/recommenders.git to commit cd41f95515d297f1b958e36a6e569fedcc82a573
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 64.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.4/316.4 kB 31.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 55.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 88.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 56.1 MB/s eta 0:00:00
  Preparing met

In [ ]:
import os
import sys
import cornac
import pandas as pd

from recommenders.datasets.python_splitters import python_random_split
from recommenders.evaluation.python_evaluation import map, ndcg_at_k, precision_at_k, recall_at_k
from recommenders.models.cornac.cornac_utils import predict_ranking
from recommenders.utils.timer import Timer
from recommenders.utils.constants import SEED


print(f"System version: {sys.version}")
print(f"Cornac version: {cornac.__version__}")

System version: 3.10.12 (main, Nov 20 2023, 15:14:05) [GCC 11.4.0]
Cornac version: 1.18


In [ ]:
# Model parameters
NUM_FACTORS = 200
NUM_EPOCHS = 100
LEARNING_RATE = 0.001

In [ ]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')
train_path = '/content/drive/MyDrive/SMU_MITB_RS/Project1/cs608_ip_train_v3.csv'
test_path =  '/content/drive/MyDrive/SMU_MITB_RS/Project1/cs608_ip_probe_v3.csv'

import pandas as pd

# Read the file into a pandas DataFrame
df_train = pd.read_csv(train_path)
df_test = pd.read_csv(test_path)
# Combine the two df for ratiosplit
df = pd.concat([df_train, df_test], axis=0, ignore_index=True)

# Display the DataFrame
print(df)

Mounted at /content/drive
        user_id  item_id  rating
0         12108        1       2
1         12108        2       5
2         12108        3       4
3         12108        4       4
4         12108        5       5
...         ...      ...     ...
281316    12568     1511       3
281317    12568    12389       4
281318    14361    18535       5
281319    14361    16572       5
281320    14361    19047       5

[281321 rows x 3 columns]


In [ ]:
print(df.dtypes)

user_id    int64
item_id    int64
rating     int64
dtype: object


In [ ]:
train_set = cornac.data.Dataset.from_uir(df.itertuples(index=False))

print('Number of users: {}'.format(train_set.num_users))
print('Number of items: {}'.format(train_set.num_items))

Number of users: 21124
Number of items: 31203


In [ ]:
from cornac.hyperopt import GridSearch,Discrete
import torch
from cornac.eval_methods import RatioSplit
from cornac.metrics import Recall, NDCG, NCRR


ratio_split = RatioSplit(data=df.values, test_size=0.1, val_size=0.1, rating_threshold=4.0, seed=2024)
param_grid =[
    Discrete(name = 'k',values =[50, 100] ),
    Discrete(name = 'max_iter',values=[50,100,200] ),
    Discrete(name = 'learning_rate',values =[0.001,0.01,0.1]),
    Discrete(name = 'lambda_reg',values = [0.001,0.01,0.1]),
]

bpr = cornac.models.BPR(
    k=50,
    max_iter=200,
    learning_rate=0.05,
    lambda_reg=0.001,
    verbose=True,
    seed=SEED
)

# Initialize GridSearch
gs= GridSearch(model=bpr,space = param_grid,metric = Recall(k=50),eval_method = ratio_split)

# Run the grid search
cornac.Experiment(
    eval_method=ratio_split,
    models=[gs],
    metrics = [NDCG(k=50),NCRR(k=50),Recall(k=50)] , # 添加 NDCG 指标
    user_based=False
).run()

print("Best parameters found:", gs.best_params)


Evaluating: {'k': 50, 'lambda_reg': 0.001, 'learning_rate': 0.001, 'max_iter': 50}


  0%|          | 0/50 [00:00<?, ?it/s]

Optimization finished!
Evaluating: {'k': 50, 'lambda_reg': 0.001, 'learning_rate': 0.001, 'max_iter': 100}


  0%|          | 0/100 [00:00<?, ?it/s]

Optimization finished!
Evaluating: {'k': 50, 'lambda_reg': 0.001, 'learning_rate': 0.001, 'max_iter': 200}


...

  0%|          | 0/100 [00:00<?, ?it/s]

Optimization finished!
Evaluating: {'k': 100, 'lambda_reg': 0.1, 'learning_rate': 0.1, 'max_iter': 200}


  0%|          | 0/200 [00:00<?, ?it/s]

Optimization finished!
Best parameter settings: {'k': 100, 'lambda_reg': 0.01, 'learning_rate': 0.1, 'max_iter': 200}
Recall@50 = 0.1213

VALIDATION:
...
               | NCRR@50 | NDCG@50 | Recall@50 | Time (s)
-------------- + ------- + ------- + --------- + --------
GridSearch_BPR |  0.0199 |  0.0415 |    0.1213 |  74.2757

TEST:
...
               | NCRR@50 | NDCG@50 | Recall@50 | Train (s) | Test (s)
-------------- + ------- + ------- + --------- + --------- + --------
GridSearch_BPR |  0.0196 |  0.0416 |    0.1228 | 4684.1858 |  80.1860

Best parameters found: {'k': 100, 'lambda_reg': 0.01, 'learning_rate': 0.1, 'max_iter': 200}


AttributeError: 'GridSearch' object has no attribute 'best_result'

In [ ]:

bpr = cornac.models.BPR(
    k=100,
    max_iter=200,
    learning_rate=0.1,
    lambda_reg=0.01,
    verbose=True,
    seed=SEED
)

In [ ]:
ratio_split = RatioSplit(data=df.values, test_size=0.1, rating_threshold=4.0, seed=2024)
metrics = [NDCG(k=50),NCRR(k=50),Recall(k=50)]

experiment = Experiment(eval_method=ratio_split, models=[bpr], metrics=metrics)
experiment.run()

  0%|          | 0/200 [00:00<?, ?it/s]

Optimization finished!

TEST:
...
    | NCRR@50 | NDCG@50 | Recall@50 | Train (s) | Test (s)
--- + ------- + ------- + --------- + --------- + --------
BPR |  0.0209 |  0.0438 |    0.1288 |   30.3106 |  81.8116



In [ ]:
import numpy as np
# Retrieve the top 50 recommendations for each user
train_user_ids = ratio_split.train_set.user_ids
test_user_ids = ratio_split.test_set.user_ids
all_user_ids = np.unique(np.concatenate((train_user_ids, test_user_ids)))

# sort the result by user_Ids
sorted_user_ids = np.sort(all_user_ids)


output_file_path = 'all_user_top_50_recommendations_sorted.txt'
with open(output_file_path, 'w') as file:
    for user_id in sorted_user_ids:
        item_recommendations = bpr.recommend(user_id, k=50)
        recommendation_line = ' '.join(str(item_id) for item_id in item_recommendations)
        file.write(recommendation_line + '\n')



In [ ]:
print(sorted_user_ids.shape)

(21124,)


In [ ]:
import os
print(os.getcwd())


/content


In [ ]:
!ls


all_user_top_50_recommendations_sorted.txt  CornacExp-2024-05-18_04-07-47-581659.log  sample_data
CornacExp-2024-05-18_03-37-57-651603.log    drive


In [ ]:
from google.colab import files
files.download('all_user_top_50_recommendations_sorted.txt')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>